In [ ]:
import pandas as pd
from sqlalchemy import create_engine
# Create conexion with PostgreSQL use SQLAlchemy
engine = create_engine('postgresql+psycopg2://postgres:_____________/liquor_oiwa')
import os 
import gc
from dotenv import load_dotenv
load_dotenv()
postgres_user= os.getenv("POSTGRES_USER")
postgres_key= os.getenv("POSTGRES_KEY")
import psycopg2


In [3]:
df = pd.read_parquet("../../../SALES_ANALYTIC/data_transformation/dataset_cleaned.parquet")

In [30]:
df.rename({'invoice/item_number':'invoice'},axis= 1, inplace= True)

In [2]:
dicc_table={
    ''' CREATE TABLE "Category"(
                        "id" INT NOT null PRIMARY KEY,
                        "name" VARCHAR(100))''',
    ''' CREATE TABLE "Vendor"(
                        "id" INT NOT null PRIMARY KEY,
                        "name" VARCHAR(100))''',
    ''' CREATE TABLE "county"(
                        "id" INT NOT null PRIMARY KEY,
                        "name" VARCHAR(100))''',
    ''' CREATE TABLE "Store"(
                        "id" INT NOT null PRIMARY KEY,
                        "name" VARCHAR(100))''',
    ''' CREATE TABLE "City"(
                        "id" INT NOT null PRIMARY KEY,
                        "name" VARCHAR(100))''',
    ''' CREATE TABLE "Item"(
                        "id" INT NOT null PRIMARY KEY,
                        "name" VARCHAR(100)),
                        "id_category" INT,
                        CONSTRAINT "fk_category"
                            FOREIGN KEY ("id_category")
                            REFERENCE "Category"("id")
                            ''',
    ''' CREATE TABLE "LOCATION"(
                        "id" INT NOT null PRIMARY KEY,
                        "id_store" INT,
                        CONSTRAINT "fk_store"
                            FOREIGN KEY ("id_store")
                            REFERENCE "Store"("id"),
                        "id_addres" INT,
                        CONSTRAINT "fk_address"
                            FOREIGN KEY ("id_address")
                            REFERENCE "Address"("id"),
                        "id_city" INT,
                        CONSTRAINT "fk_city"
                            FOREIGN KEY ("id_city")
                            REFERENCE "City"("id"),
                        "zip_code" INT,
                        "latitude" NUMERICAL,
                        "longitude" NUMERICAL
                            ''',
    ''' CREATE TABLE "INVOICE"(
                        "id" INT NOT null PRIMARY KEY,
                        "invoice" VARCHAR (100),
                        "id_location" INT,
                        CONSTRAINT "fk_location"
                            FOREIGN KEY ("id_location")
                            REFERENCE "Location"("id"),
                        "date" timestamp,
                        "id_item" INT,
                        CONSTRAINT "fk_item"
                            FOREIGN KEY ("id_item")
                            REFERENCE "Item"("id"),
                        "pack" INT,
                        "bottle_volume(ml)" INT,
                        "state_bottle_cost" NUMERICAL,
                        "state_bottle_retail" NUMERICAL,
                        "bottles_sold" INT,
                        "sale(dollars)" NUMERICAL,
                        "volume_sold(ml)" NUMERICAL,
                        "id_vendor" INT,
                        CONSTRAINT "fk_vendor"
                            FOREIGN KEY ("id_vendor")
                            REFERENCE "Vendor"("id"),
                        "id_county" INT,
                        CONSTRAINT "fk_county"
                            FOREIGN KEY ("id_county")
                            REFERENCE "County"("id")
                            '''

   
}

In [ ]:
# Conect databasee
conn = psycopg2.connect(
    host='localhost',
    database='liquor_oiwa',
    user='postgres',
    password='____________'
)
conn.autocommit = True

# Create new cursor
cursor = conn.cursor()

#create  table with   dicc_table
for create_stmt in dicc_table:
    try:
        cursor.execute(create_stmt)
        print("Tabla creada correctamente:\n", create_stmt.split('(')[0])
    except Exception as e:
        print("Error al crear tabla:\n", create_stmt.split('(')[0])
        print(e)

# Close cursor with database
cursor.close()
conn.close()

In [ ]:
# insert dataframe with database 
df.to_sql('INVOICE', con=engine, if_exists='append', index=False)
df.to_sql('county', con=engine, if_exists='append', index=False)
df.to_sql('LOCATION', con=engine, if_exists='append', index=False)
df.to_sql('Item', con=engine, if_exists='append', index=False)
df.to_sql('City', con=engine, if_exists='append', index=False)
df.to_sql('Vendor', con=engine, if_exists='append', index=False)
df.to_sql('Category', con=engine, if_exists='append', index=False)



In [ ]:

df_store = df[['store_name']].drop_duplicates()

# renema the colums'store_name' a 'name' 
df_store.rename(columns={'store_name': 'name'}, inplace=True)

# Insert 'Store'
df.to_sql('county', con=engine, if_exists='append', index=False)